<a href="https://colab.research.google.com/github/amgothhrithik/Search_Engine/blob/main/image_search_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset =  load_dataset(
    "ashraq/fashion-product-images-small",
    split="train" )


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/867 [00:00<?, ?B/s]

(…)-00000-of-00002-6cff4c59f91661c3.parquet:   0%|          | 0.00/136M [00:00<?, ?B/s]

(…)-00001-of-00002-bb459e5ac5f01e71.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44072 [00:00<?, ? examples/s]

In [ ]:
print(dataset.features)

{'id': Value(dtype='int64', id=None), 'gender': Value(dtype='string', id=None), 'masterCategory': Value(dtype='string', id=None), 'subCategory': Value(dtype='string', id=None), 'articleType': Value(dtype='string', id=None), 'baseColour': Value(dtype='string', id=None), 'season': Value(dtype='string', id=None), 'year': Value(dtype='float64', id=None), 'usage': Value(dtype='string', id=None), 'productDisplayName': Value(dtype='string', id=None), 'image': Image(mode=None, decode=True, id=None)}


In [ ]:
print(dataset[100])

{'id': 19311, 'gender': 'Men', 'masterCategory': 'Footwear', 'subCategory': 'Shoes', 'articleType': 'Casual Shoes', 'baseColour': 'White', 'season': 'Fall', 'year': 2011.0, 'usage': 'Casual', 'productDisplayName': 'Nike Men Air Rift MTR White Casual Shoe', 'image': <PIL.Image.Image image mode=RGB size=60x80 at 0x7CFFFB3126D0>}


In [ ]:
from transformers import CLIPProcessor, CLIPModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


In [ ]:
images=dataset['image']

In [ ]:
caption=dataset['productDisplayName']

In [ ]:
from PIL import Image
from tqdm import tqdm
image_embeddings, text_embeddings = [], []
batch_size = 32

for i in tqdm(range(0, len(dataset), batch_size)):
    images_batch = images[i:i+batch_size]
    texts_batch = caption[i:i+batch_size]

    # Prepare inputs separately for image and text
    image_inputs = processor(images=images_batch, return_tensors="pt").to(device)
    text_inputs = processor(text=texts_batch, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        img_embeds = model.get_image_features(**image_inputs)
        txt_embeds = model.get_text_features(**text_inputs)

    image_embeddings.append(img_embeds.cpu())
    text_embeddings.append(txt_embeds.cpu())

# Combine and average
image_embeddings = torch.cat(image_embeddings, dim=0)
text_embeddings = torch.cat(text_embeddings, dim=0)

final_embeddings = 0.5 * image_embeddings + 0.5 * text_embeddings
final_embeddings_np = final_embeddings.numpy().astype("float32")

100%|██████████| 1378/1378 [04:48<00:00,  4.78it/s]


In [ ]:
final_embeddings.shape

torch.Size([44072, 512])

In [ ]:
import numpy as np

np.save("/content/drive/MyDrive/Data/search_engine/embeddings.npy", final_embeddings.cpu().numpy())  # Move to CPU before saving
